# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

**Simon G.**

**30 May 2020**

---

## **Part 1 of the assignment**
### *loading data*

In [1]:
#import libraries and packages
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#!conda install -c conda-forge lxml --yes

import json 
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Get pandas to read webpage

Further drop values where we do not have any value for Borough and reset the index

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df_raw = pd.read_html(url)
df = pd.DataFrame(df_raw[0])
df_cleaned = df[~df['Borough'].isin(['Not assigned'])].reset_index(drop=True)
print ('We have {} unique Postal codes' .format(len(df_cleaned['Postal Code'].unique())))
df_cleaned.head(10)

We have 103 unique Postal codes


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Shape of the dataframe

In [3]:
df_cleaned.shape

(103, 3)

---

## **Part 2 of the assignment**

### *adding coordinates*

Get csv since geocoder is loading for ever..


In [4]:
url = 'http://cocl.us/Geospatial_data'

df_latlon = pd.read_csv(url)
df_latlon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
df_merged = df_cleaned.join(df_latlon.set_index('Postal Code'), on='Postal Code')
df_merged.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


---

## Part 3 of the assignment

### *clustering*

In [6]:
df_to_all = df_merged[df_merged['Borough'].str.contains('Toronto')].reset_index(drop=True)
df_to_all.head(5)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [7]:
df_to = df_to_all.drop(['Borough', 'Neighborhood'], axis=1)
df_to.head()

,Postal Code,Latitude,Longitude
0,M5A,43.654260,-79.360636
1,M7A,43.662301,-79.389494
2,M5B,43.657162,-79.378937
3,M5C,43.651494,-79.375418
4,M4E,43.676357,-79.293031


### Foursquare Credentials and Version

In [8]:
CLIENT_ID = 'QAXHZLGZ1B3LIEE5TWIXLLRB254ZZO1GTOYEPXZRLLLPS43E' # your Foursquare ID
CLIENT_SECRET = 'RLBEV2B4C3Q3F3S1IKSO1SAHE1OY4PA1CXPA0JLGQ0DMTBLF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QAXHZLGZ1B3LIEE5TWIXLLRB254ZZO1GTOYEPXZRLLLPS43E
CLIENT_SECRET:RLBEV2B4C3Q3F3S1IKSO1SAHE1OY4PA1CXPA0JLGQ0DMTBLF


In [9]:
latitude = 43.6532
longitude = -79.3832
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=QAXHZLGZ1B3LIEE5TWIXLLRB254ZZO1GTOYEPXZRLLLPS43E&client_secret=RLBEV2B4C3Q3F3S1IKSO1SAHE1OY4PA1CXPA0JLGQ0DMTBLF&ll=43.6532,-79.3832&v=20180605&radius=500&limit=100'

In [10]:
results = requests.get(url).json()

In [11]:
def getNearbyVenues(postal_code, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for postal_code, lat, lng in zip(postal_code, latitudes, longitudes):
    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        try:
            venues_list.append([(
                postal_code, 
                lat, 
                lng,
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        except:
            pass

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal_code', 
                  'PC Latitude', 
                  'PC Longitude', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
# type your answer here

toronto_venues = getNearbyVenues(postal_code=df_to['Postal Code'],
                                   latitudes=df_to['Latitude'],
                                   longitudes=df_to['Longitude']
                                  )

print (toronto_venues.shape)
toronto_venues.head()

(1616, 6)


,Postal_code,PC Latitude,PC Longitude,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65426,-79.360636,43.653447,-79.362017,Bakery
1,M5A,43.65426,-79.360636,43.653559,-79.361809,Coffee Shop
2,M5A,43.65426,-79.360636,43.653947,-79.361149,Breakfast Spot
3,M5A,43.65426,-79.360636,43.653249,-79.358008,Distribution Center
4,M5A,43.65426,-79.360636,43.654735,-79.359874,Spa


In [13]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


## Prep data for clustering

In [14]:
#one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="") 
toronto_onehot['Postal_code'] = toronto_venues['Postal_code']
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print (toronto_onehot.shape)
toronto_onehot.head()

(1616, 237)


,Postal_code,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [15]:
#group by postal code and calculate mean
toronto_grouped = toronto_onehot.groupby('Postal_code').mean().reset_index()
print (toronto_grouped.shape)
toronto_grouped.head()

(39, 237)


,Postal_code,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4E,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.00000,0.0,0.0,0.1

In [16]:
#function to get the highest values
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
#use function above to loop through the rows
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal_code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
pc_venues_sorted = pd.DataFrame(columns=columns)
pc_venues_sorted['Postal_code'] = toronto_grouped['Postal_code']

for ind in np.arange(toronto_grouped.shape[0]):
    pc_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

pc_venues_sorted.head()

,Postal_code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,Park,Pub,Coffee Shop,Health Food Store,Neighborhood
1,M4K,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop
2,M4L,Sandwich Place,Fast Food Restaurant,Pizza Place,Sushi Restaurant,Pub
3,M4M,Café,Coffee Shop,Gastropub,American Restaurant,Bakery
4,M4N,Park,Bus Line,Swim School,Yoga Studio,Dessert Shop


In [18]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postal_code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# add clustering labels
pc_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [19]:
# merging the former df with the sorted one
toronto_merged = df_to.join(pc_venues_sorted.set_index('Postal_code'), on='Postal Code')
toronto_merged.head() 

,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,43.654260,-79.360636,2,Coffee Shop,Pub,Bakery,Park,Café
1,M7A,43.662301,-79.389494,2,Coffee Shop,Bank,Beer Bar,Smoothie Shop,Sandwich Place
2,M5B,43.657162,-79.378937,2,Clothing Store,Coffee Shop,Japanese Restaurant,Restaurant,Bubble Tea Shop
3,M5C,43.651494,-79.375418,2,Coffee Shop,Café,Cocktail Bar,American Restaurant,Restaurant
4,M4E,43.676357,-79.293031,0,Park,Pub,Coffee Shop,Health Food Store,Neighborhood


---

## Create the map

In [20]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
#x = np.arange(kclusters)
#ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]
color = ['red', 'blue', 'green', 'purple', 'orange']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.Marker(location=[lat, lon],popup=label,icon=folium.Icon(color=color[cluster-1], icon='circle')).add_to(map_clusters)

### validate the cluster labels
1. has many social places is close to the city center, therefore it is classified as **close to city center**
2. Is further away from the city center and appears like a living area, therefore it is classified as **residental area**
3. Most Coffee Shops in this area, therefore it is classified as **city center**
4. Park and more outside, therefore it is classified as **external park**
5. Parks and playgrounds but closer to the city center, therefore classified as **city parks**

In [21]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,M4E,Park,Pub,Coffee Shop,Health Food Store,Neighborhood
7,M6G,Grocery Store,Café,Park,Candy Store,Baby Store
9,M6H,Bakery,Pharmacy,Smoke Shop,Music Venue,Supermarket
12,M4K,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop
14,M6K,Café,Breakfast Spot,Nightclub,Coffee Shop,Furniture / Home Store
15,M4L,Sandwich Place,Fast Food Restaurant,Pizza Place,Sushi Restaurant,Pub
17,M4M,Café,Coffee Shop,Gastropub,American Restaurant,Bakery
22,M6P,Arts & Crafts Store,Thai Restaurant,Café,Mexican Restaurant,Furniture / Home Store
24,M5R,Sandwich Place,Café,Pharmacy,Coffee Shop,BBQ Joint
26,M4S,Dessert Shop,Sandwich Place,Café,Sushi Restaurant,Gym


In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
19,M5N,Ice Cream Shop,Home Service,Garden,Department Store,Falafel Restaurant


In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Coffee Shop,Pub,Bakery,Park,Café
1,M7A,Coffee Shop,Bank,Beer Bar,Smoothie Shop,Sandwich Place
2,M5B,Clothing Store,Coffee Shop,Japanese Restaurant,Restaurant,Bubble Tea Shop
3,M5C,Coffee Shop,Café,Cocktail Bar,American Restaurant,Restaurant
5,M5E,Coffee Shop,Cocktail Bar,Seafood Restaurant,Café,Pub
6,M5G,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Bubble Tea Shop
8,M5H,Coffee Shop,Café,Restaurant,Clothing Store,Gym
10,M5J,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant
11,M6J,Bar,Restaurant,Asian Restaurant,Café,Vegetarian / Vegan Restaurant
13,M5K,Coffee Shop,Café,Hotel,Restaurant,Salad Place


In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
18,M4N,Park,Bus Line,Swim School,Yoga Studio,Dessert Shop


In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
21,M5P,Trail,Park,Sushi Restaurant,Jewelry Store,Doner Restaurant
33,M4W,Park,Playground,Trail,Yoga Studio,Deli / Bodega


### Plot the map with the legend

In [26]:
legend_html =   '''
                <div style="position: fixed; 
                            top: 100px; left: 100px; width: 200px; height: 200px; 
                            border:4px solid grey; z-index:9999; font-size:14px;
                            ">&nbsp; Legend <br>
                              &nbsp; <i class="fa fa-map-marker fa-2x" style="color:orange"></i> close to city center &nbsp; <br>
                              &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i> residential area &nbsp; <br>
                              &nbsp; <i class="fa fa-map-marker fa-2x" style="color:blue"></i> city center &nbsp; <br>
                              &nbsp; <i class="fa fa-map-marker fa-2x" style="color:green"></i> external parks &nbsp; <br>
                              &nbsp; <i class="fa fa-map-marker fa-2x" style="color:purple"></i> city parks &nbsp; <br>
                              
                </div>
                ''' 

map_clusters.get_root().html.add_child(folium.Element(legend_html))

map_clusters